In [ ]:
### Compute the $\log N - \log F$ of the individual pulses of BATSE simulated LCs 
For each pulse of every simulated LC, we convert the peak count rates into peak fluxes through a conversion factors, $k$. These factors account for the different spectral properties of the GRBs. If we call $C$ the peak count rate and $F$ the peak flux, the latter will be:
$F = 10^k C$. 
################################################################################
# Load the conversion factors: they are computed in the 25-2000 keV energy band
################################################################################

path_k_values = '../lc_pulse_avalanche/log10_fluence_over_counts_CGRO_BATSE.txt'
k_values = np.loadtxt(path_k_values)

################################################################################
# Plot the distribution of the conversion factors
################################################################################

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.hist(k_values, bins=np.linspace(np.min(k_values), np.max(k_values), 20))
ax.set_xlabel(r'$k_{25-2000~\mathrm{keV}}$', size=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.grid(which='both')
plt.show()
################################################################################
# Load the peak count rates
################################################################################

lc_path = '/home/manuele/Documents/university/grbs/geneticgrbs_simulations/batse_lc_params/'
lc_list = os.listdir(lc_path)
lc_list.sort()
# lc_list

peak_count_rates, pulse_counts = np.loadtxt(lc_path + lc_list[0], ndmin=2, usecols=(0, 4), unpack=True)
for lc in lc_list[1:]:
    peak_count_rate, pulse_count = np.loadtxt(lc_path + lc, ndmin=2, usecols=(0, 4), unpack=True)
    peak_count_rates = np.concatenate((peak_count_rates, peak_count_rate))
    pulse_counts = np.concatenate((pulse_counts, pulse_count))
peak_count_rates.sort()
pulse_counts.sort()
# peak_count_rates
len(peak_count_rates) == len(pulse_counts)
len(pulse_counts)
################################################################################
# Convert peak count rates to peak fluxes
################################################################################

peak_fluxes = np.zeros_like(peak_count_rates)
for i in range(len(peak_fluxes)):
    k = np.random.choice(k_values)
    peak_fluxes[i] = (10.**k)*peak_count_rates[i]
peak_fluxes.sort()
# peak_fluxes
################################################################################
# BATSE broken power law parameters obtained through the GA
################################################################################

alpha_bpl = 1.61
beta_bpl  = 2.19
F_break   = 6.18e-07
F_min     = 4.87e-08

################################################################################
# Plot the distribution of the logarithm of the peak fluxes
################################################################################

def BrokenPowerLaw(x, alpha, beta, x_0, x_min):
    N = (x_0*((1-(x_min/x_0)**(1-alpha))/(1-alpha)-1/(1-beta)))**(-1)
    return np.piecewise(x, [x < x_0, x >= x_0], [lambda x: N*(x/x_0)**(-alpha), 
                                                 lambda x: N*(x/x_0)**(-beta)])

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
log_bins = np.logspace(min(np.log10(peak_fluxes)), max(np.log10(peak_fluxes)), 100)
ax.hist(peak_fluxes, bins=log_bins, density=True)
ax.plot(peak_fluxes, BrokenPowerLaw(peak_fluxes, alpha_bpl, beta_bpl, F_break, F_min), c='r', lw=3)
ax.set_xlabel(r'$\log F_{25-2000~\mathrm{keV}}$', size=18)
ax.set_ylabel(r'$\log N$', size=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.grid(which='both')
ax.loglog()
plt.show()
### Compute the $\log N - \log S$ of the individual pulses
################################################################################
# Convert the pulse counts into pulse fluence
################################################################################

# There are some pulses for which the total number of counts is equal to zero:
# this happens because they occur outside the 150 s considered by the algorithm.

pulse_counts = np.array(pulse_counts)
pulse_counts = pulse_counts[pulse_counts > 1.e-05]

pulse_fluences = np.zeros_like(pulse_counts)
for i in range(len(pulse_fluences)):
    k = np.random.choice(k_values)
    pulse_fluences[i] = (10.**k)*pulse_counts[i]
pulse_fluences.sort()
# for i in range(len(pulse_fluences)):
#     print(pulse_fluences[i])
len(pulse_fluences)
################################################################################
# Plot the fluence distribution
################################################################################

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
log_bins = np.logspace(min(np.log10(pulse_fluences)), max(np.log10(pulse_fluences)), 100)
pulse_freqs, pulse_bins, _ = ax.hist(pulse_fluences, bins=log_bins, density=True)
ax.set_xlabel(r'$\log S_{25-2000~\mathrm{keV}}$', size=18)
ax.set_ylabel(r'$\log N$', size=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.grid(which='both')
ax.loglog()
plt.show()
# Compute the survival function starting from the cumulative one 
pulse_cdf = np.cumsum(pulse_freqs*np.diff(pulse_bins))
pulse_srv = 1 - pulse_cdf

# Plot the survival function
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.step(log_bins[:-1], pulse_srv, c='b', lw=3)
ax.set_xlabel(r'$\log S_{25-2000~\mathrm{keV}}$', size=18)
ax.set_ylabel(r'$\log N$', size=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.set_ylim(1.e-05, 2.)
ax.loglog()
ax.grid(which='both')
plt.show()
### Compute the $\log N-\log S$ of the BATSE GRBs
# Load the fluences of BATSE GRBs: they are computed in the four BATSE energy bands
fluences1, fluences2, fluences3, fluences4 = np.loadtxt('./flux_1row.txt', usecols=(1, 3, 5, 7), unpack=True)

# Compute the fluence in the total BATSE energy band (25-2000 keV)
total_fluences = fluences1 + fluences2 + fluences3 + fluences4
# Sanity check: check whether there is any fluence = 0
total_fluences.sort()
for i in range(len(total_fluences)):
    if total_fluences[i] == 0:
        print('%4d %7.2e' % (i, total_fluences[i]))
len(total_fluences)
# Plot the distribution of the logarithm of the fluence
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
log_bins = np.logspace(min(np.log10(total_fluences[1:])), max(np.log10(total_fluences[1:])), 100)
burst_freqs, burst_bins, _ = ax.hist(total_fluences[1:], bins=log_bins, density=True)
ax.set_xlabel(r'$\log S_{25-2000~\mathrm{keV}}$', size=18)
ax.set_ylabel(r'$\log N$', size=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.loglog()
ax.grid(which='both')
plt.show()
# Compute the survival function starting from the cumulative one 
burst_cdf = np.cumsum(burst_freqs*np.diff(burst_bins))
burst_srv = 1 - burst_cdf

# Plot the survival function
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.step(burst_bins[:-1], burst_srv, c='b', lw=3)
ax.set_xlabel(r'$\log S_{25-2000~\mathrm{keV}}$', size=18)
ax.set_ylabel(r'$\log N$', size=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.set_ylim(1.e-05, 2.)
ax.loglog()
plt.show()
# Compare the two survival functions
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.step(pulse_bins[:-1], pulse_srv, c='b', lw=3, label='Pulses')
ax.step(burst_bins[:-1], burst_srv, c='r', lw=3, label='GRBs')
ax.set_xlabel(r'$\log S_{25-2000~\mathrm{keV}}$', size=18)
ax.set_ylabel(r'$\log N$', size=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.set_ylim(1.e-05, 2.)
ax.loglog()
ax.grid(which='both')
ax.legend(fontsize=14)
plt.show()
### Computing the BAT's fluence distribution from the BATSE's one
- 1) We randomly sample a fluence value $F$ from the BATSE's fluence distribution that we obtained through the genetic algorithm.
- 2) We randomly sample a value for the spectral parameters $\alpha$, $\beta$, and $E_{\mathrm{peak}}$ of the Band's function from the corresponding distributions that are found in the most recent BATSE's spectral catalogue (Goldstein et al. 2013). 
- 3) Fixing the amplitude to 1, we integrate the spectrum in the BAT ($15-150$ keV) and BATSE ($50-300$ keV) energy ranges. The ratio between the BAT and BATSE photon rates, $P_{\mathrm{BAT}}$ and $P_{\mathrm{BATSE}}$, gives us the rescaling factor $R = P_{\mathrm{BAT}}/P_{\mathrm{BATSE}}$ that multiplies $F$. 
Load the BATSE's spectral catalogue fitted with Band function
with open('./batse_grb_spectral_catalogue.txt', 'r') as f:
    lines = f.readlines()

# Select only those lines that refer to the fluence spectra
spectral_table = lines[32:2177]

# Print the first and the last lines of the table: check that they correspond 
# to those in the file
# print(spectral_table[0])
# print(spectral_table[-1])

# Save the table into a file
np.savetxt('./grb_fluence_spectra_fit_band.txt', spectral_table, fmt='%s')
Load the table of BATSE GRB fluence spectra fitted with Band function
alpha, e_alpha, \
beta,  e_beta,  \
Epeak, e_Epeak = np.loadtxt('./grb_fluence_spectra_fit_band.txt', usecols=(6, 7, 8, 9, 10, 11), unpack=True)

import pandas as pd
spectral_dict = {'alpha': alpha, 'e_alpha': e_alpha, 
                 'beta' : beta,  'e_beta' : e_beta,
                 'Epeak': Epeak, 'e_Epeak': e_Epeak}

spectral_df = pd.DataFrame(spectral_dict)
spectral_df
We select only those GRBs for which:
- $\delta\alpha < 0.4$;
- $\delta\beta < 1.0$;
- $\dfrac{\delta E_{\mathrm{peak}}}{E_{\mathrm{peak}}} < 0.4$;
good_spectral_df = spectral_df[
    (spectral_df['alpha'] < 0) & (spectral_df['e_alpha'] != 0) & (spectral_df['e_alpha'] < 0.4) & 
    (spectral_df['beta']  < 0) & (spectral_df['e_beta']  != 0) & (spectral_df['e_beta']  < 1.0) &
    (spectral_df['e_Epeak'] != 0) & (spectral_df['e_Epeak']/spectral_df['Epeak'] < 0.4)
]
good_alpha = good_spectral_df['alpha']
good_beta  = good_spectral_df['beta' ]
good_Epeak = good_spectral_df['Epeak']
Plot the spectral parameters' distributions
fig, ax = plt.subplots(1, 3, figsize=(24, 6))
# Plot 1
ax[0].hist(good_alpha, bins=np.linspace(good_alpha.min(), good_alpha.max(), 20))
ax[0].set_xlabel(r'$\alpha$', size=18)
ax[0].xaxis.set_tick_params(labelsize=18)
ax[0].yaxis.set_tick_params(labelsize=18)
ax[0].grid()
# Plot 1
ax[1].hist(good_beta, bins=np.linspace(good_beta.min(), good_beta.max(), 20))
ax[1].set_xlabel(r'$\beta$', size=18)
ax[1].xaxis.set_tick_params(labelsize=18)
ax[1].yaxis.set_tick_params(labelsize=18)
ax[1].grid()
# Plot 1
ax[2].hist(good_Epeak, bins=np.logspace(np.log10(good_Epeak.min()), np.log10(good_Epeak.max()), 20))
ax[2].set_xlabel(r'$E_{\mathrm{peak}}$ (keV)', size=18)
ax[2].xaxis.set_tick_params(labelsize=18)
ax[2].yaxis.set_tick_params(labelsize=18)
ax[2].set_xscale('log')
ax[2].grid()
plt.show()
Band's function
def Band(E, E_peak, alpha, beta):
    """Band’s GRB function (Band et al. 1993) for fitting GRB spectra.

    Args:
        E      (float): energy;
        E_peak (float): peak energy;
        alpha  (float): low-energy index;
        beta   (float): high-energy index.
        
    Returns:
        float: the corresponding flux.
    """
    # Amplitude: set equal to 1 (ph/s/cm^2/keV)
    A   = 1.
    E_0 = (alpha - beta)*E_peak/(alpha + 2)
    return np.piecewise(E, [E >= E_0, E < E_0], [lambda E: E * A * ((E/100.)**alpha) * (np.exp(- (alpha + 2) * E / E_peak)),
                                                 lambda E: E * A * ((E/100.)**beta)  * (np.exp(beta - alpha)) * ((alpha - beta) * E_peak / (100. * (alpha + 2)))**(alpha - beta)])
BATSE's peak flux distribution
def generate_flux(p, alpha, beta, F_break, F_min):
    """This function returns a peak flux value F from the broken-power-law (BPL)
    distribution: 
    
    f(F) = N * (F/F_break)**(-alpha) if F <  F_break;
           N * (F/F_break)**(-beta)  if F >= F_break;
    
    where alpha and beta are BPL indices, F_break is the break, and F_min is 
    minimum peak flux. The idea is sampling the CDF, whose values are found 
    between 0 and 1 by definition, and turn them into the corresponding 
    peak flux values.

    Args:
        p       (float): sampled value from the CDF;
        alpha   (float): first BPL index;
        beta    (float): second BPL index;
        F_break (float): break peak flux;
        F_min   (float): minimum peak flux.

    Returns:
        float: the corresponding peak flux value F.
    """
    R_min = F_min/F_break
    f_0   = (F_break*((1-R_min**(1-alpha))/(1-alpha)-1/(1-beta)))**(-1)
    p_0   = f_0*F_break*(1-R_min**(1-alpha))/(1-alpha)
    return np.piecewise(p, [p < p_0, p >= p_0], [lambda p: F_break*(p*(1-alpha)/(f_0*F_break)+R_min**(1-alpha))**(1/(1-alpha)), 
                                                 lambda p: F_break*((p-1)*(1-beta)/(f_0*F_break))**(1/(1-beta))])
Parameters of the BATSE's fluence distribution that we obtained through the genetic algorithm
alpha_bpl_batse = 1.61
beta_bpl_batse  = 2.19
F_break_batse   = 6.18e-07
F_min_batse     = 4.87e-08
Parameters of the BAT's fluence distribution that we obtained through the genetic algorithm
alpha_bpl_bat = 1.89
beta_bpl_bat  = 2.59
F_break_bat   = 1.67e-07
F_min_bat     = 1.01e-08
Integrate the Band's function in the BAT and BATSE energy ranges and rescaling the fluence 
from scipy import integrate

F_bat   = np.zeros(10000)
F_batse = np.zeros(10000)
R       = np.zeros(10000)

for i in range(len(F_bat)):
    # Randomly sample the BATSE's peak flux distribution
    F_batse[i] = generate_flux(np.random.rand(), alpha_bpl_batse, beta_bpl_batse, F_break_batse, F_min_batse)
    
    # Randomly sample the spectral parameters
    sampled_alpha = np.random.choice(good_alpha)
    sampled_beta  = np.random.choice(good_beta)
    sampled_Epeak = np.random.choice(good_Epeak)

    # Integrate the spectra in the BAT (15-150 keV) and BATSE (25-2000 keV) energy ranges
    P_bat   = integrate.quad(Band, 15, 150, args=(sampled_Epeak, sampled_alpha, sampled_beta))
    P_batse = integrate.quad(Band, 25, 2000, args=(sampled_Epeak, sampled_alpha, sampled_beta))
    
    # Rescaling factor
    R[i] = P_bat[0]/P_batse[0]
    
    # BAT's peak flux
    F_bat[i] = R[i]*F_batse[i]
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.hist(R, bins=np.logspace(np.log10(R.min()), np.log10(R.max()), 20))
ax.set_xlabel(r'$R$', size=18)
ax.xaxis.set_tick_params(labelsize=18)
ax.yaxis.set_tick_params(labelsize=18)
ax.grid(which='both')
ax.loglog()
plt.show()
Plot the BAT's peak flux distribution
x_bat   = np.logspace(np.log10(np.min(F_bat)),   np.log10(np.max(F_bat)),   256)
x_batse = np.logspace(np.log10(np.min(F_batse)), np.log10(np.max(F_batse)), 256)

y_bat   = BrokenPowerLaw(x_bat,   alpha_bpl_bat,   beta_bpl_bat,   F_break_bat,   F_min_bat)
y_batse = BrokenPowerLaw(x_batse, alpha_bpl_batse, beta_bpl_batse, F_break_batse, F_min_batse)
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
counts_bat,   bins_bat,   _ = ax.hist(F_bat,   bins=np.logspace(np.log10(np.min(F_bat)),   np.log10(np.max(F_bat)),   50), 
                                      label='BAT',   alpha=0.4, density=True)
counts_batse, bins_batse, _ = ax.hist(F_batse, bins=np.logspace(np.log10(np.min(F_batse)), np.log10(np.max(F_batse)), 50), 
                                      label='BATSE', alpha=0.4, density=True)
# ax.plot(x_bat,   y_bat,   lw=3, label='BAT')
# ax.plot(x_batse, y_batse, lw=3, label='BATSE')
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.set_xlabel(r'$F$ (erg/cm$^2$)', size=18)
ax.loglog()
ax.grid()
ax.legend(fontsize=14)
plt.show()
# Compute the survival function starting from the cumulative one 
batse_cdf = np.cumsum(counts_batse*np.diff(bins_batse))
batse_srv = 1 - batse_cdf
bat_cdf   = np.cumsum(counts_bat*np.diff(bins_bat))
bat_srv   = 1 - bat_cdf

# Plot the survival function
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.step(bins_batse[:-1], batse_srv, c='b', lw=3, label='BATSE')
ax.step(bins_bat[:-1],   bat_srv,   c='r', lw=3, label='BAT')
ax.set_xlabel(r'$\log S_{25-2000~\mathrm{keV}}$', size=18)
ax.set_ylabel(r'$\log N$', size=18)
ax.xaxis.set_tick_params(labelsize=14)
ax.yaxis.set_tick_params(labelsize=14)
ax.set_ylim(1.e-05, 2.)
ax.loglog()
ax.grid()
ax.legend(fontsize=14)
plt.show()